In [ ]:
import numpy as np
import rasterio as rio
import geopandas as gpd
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from shapely.ops import unary_union
from shapely.geometry.polygon import Polygon
from cartopy.feature import ShapelyFeature
import matplotlib.patches as mpatches


In [ ]:
def percentile_stretch(img, pmin=0., pmax=100.):
    '''
    Stretch image values to the percentile range defined by pmin and pmax.
    '''
    if not 0 <= pmin < pmax <= 100:
        raise ValueError('0 <= pmin < pmax <= 100')
    if not img.ndim == 2:
        raise ValueError('Image can only have two dimensions (row, column)')

    minval = np.percentile(img, pmin)
    maxval = np.percentile(img, pmax)

    stretched = (img - minval) / (maxval - minval)
    stretched[img < minval] = 0
    stretched[img > maxval] = 1

    return stretched

In [ ]:
def img_display(img, ax, bands, stretch_args=None, **imshow_args):
    '''
    Display the image with optional stretch parameters for each band.
    '''
    dispimg = img.copy().astype(np.float32)

    for b in range(img.shape[0]):
        if stretch_args is None:
            dispimg[b] = percentile_stretch(img[b])
        else:
            dispimg[b] = percentile_stretch(img[b], **stretch_args)

    dispimg = dispimg.transpose([1, 2, 0])
    handle = ax.imshow(dispimg[:, :, bands], **imshow_args)

    return handle, ax

In [ ]:
# Open the satellite image
with rio.open('data_files/NI_Mosaic.tif') as dataset:
    img = dataset.read()
    xmin, ymin, xmax, ymax = dataset.bounds

In [ ]:
# Load the shapefiles for county boundaries, towns, and cities
counties = gpd.read_file('data_files/Counties.shp')
towns = gpd.read_file('data_files/towns.shp')


In [ ]:
# Create a figure and axis object for the map
fig, ax = plt.subplots(figsize=(10, 10), dpi=100)
ax.set_extent([xmin, xmax, ymin, ymax], crs=ccrs.PlateCarree())

In [ ]:
# Add the satellite image
img_display(img, ax, bands=[0, 1, 2], stretch_args={'pmin': 2, 'pmax': 98})

In [ ]:
# Add the county boundaries
counties_feature = ShapelyFeature(counties.geometry, ccrs.PlateCarree(), edgecolor='red', facecolor='none', linewidth=2)
ax.add_feature(counties_feature)